# 3. AB Testing for Optimal Game Design

[Cookie Cats](https://www.facebook.com/cookiecatsgame) is a popular "connect three" puzzle for mobile devices. To win the level you need to connect tiles of the same color and clear the board.  Let's watch a [demo video](https://youtu.be/GaP5f0jVTWE)

As many of the games that can be downloaded for free, this game makes a free play somewhat annoying and asks you to wait some time when you enounter a gate as you progress through the levels 
<center><img src="cc_gates.png" width="500"></center>

## We test two different designs
You can get rid of the gates by making an in-app purchase.

We want to understand where those gates are to be placed. The game maker had two options, placing after level 30 and after level 40. The goal of the game maker is to maximize the number of rounds played by each player and to increase retention. We will use those metrics and confidence intervals technique we've learned to pick the best out of those two designs. 

We start by loading libraries we need to read data from the csv file and to manipulate it.

In [1]:
# now we can read the data
d = read.csv("cookie_cats.csv");

In [2]:
head(d) # show first 6 rows

userid,version,sum_gamerounds,retention_1,retention_7
<int>,<fct>,<int>,<int>,<int>
116,gate_30,3,0,0
337,gate_30,38,1,0
377,gate_40,165,1,0
483,gate_40,1,0,0
488,gate_40,179,1,1
540,gate_40,187,1,1


-  `userid`: unique user ID
-  `version`: versions with different gates preventing the player’s progress, after 30 levels or after 40, these are recorded as gate_30 and gate_40
 - fct: factorial
- `sum_gamerounds`: number of rounds played by each unique user during the first 14 days after install
- `retention_1`:  player is still active after 1 day
- `retention_7`:  player is still active after 7 days

In [3]:
dim(d) # data has 90189 rows and 5 columns

[1] 90189     5

In [4]:
# Make sure we have two distinct versions 
unique(d$version)

[1] gate_30 gate_40
Levels: gate_30 gate_40

In [5]:
# Count number of unique users in each group
group_30 = d[d$version == "gate_30",] #rows when gate_30 users are selected
group_40 = d[d$version == "gate_40",] #rows when gate_40 users are selected
nrow(group_30) # number of rows
nrow(group_40) # number of rows

[1] 44700

[1] 45489

In [9]:
# People who have played < 100 rounds
par(mar=c(4,4,0,1))
hist(d$sum_gamerounds[d$sum_gamerounds<100], breaks=25, xlab="Game Rounds", ylab="Number of Users", main="", col="lightblue")

In [13]:
# Count average retention after 1 day by group
options(digits=1) # only show 1 significant digit when printing numbers (I think you have to use print for it to work)
mean(group_30$retention_1) # 44.8%
mean(group_40$retention_1) # 44.2%

[1] 0.4

[1] 0.4


In [11]:
# Count average retention after 7 days by group
mean(group_30$retention_7) # 19%
mean(group_40$retention_7) # 18.2%

[1] 0.2

[1] 0.2

In [12]:
# Count average numper of rounds by group
mean(group_30$sum_gamerounds) # 52.5
mean(group_40$sum_gamerounds) # 51.3

[1] 52

[1] 51

The difference between retention rates after 1 day in `gate_30` group is <font color=blue>44.8%</font> and in `gate_40` group is <font color=blue>44.2%</font>. The difference is too small to make a certain statement. 

<center>Let's calculate the confidence interval!</center>

In [22]:
# We put our code to calculate a confidence interval inside a function

# we always assume that means are normally distributed
confidence_interval = function(x) 
{
    n = length(x)
    xbar = sum(x)/n
    sigma2 = sum((x-xbar)^2)/n
    sigma = sqrt(sigma2)
    SEM = sigma/sqrt(n)

    cat(sprintf("Number of samples is %d\n", n))
    cat(sprintf("xbar is %.2f\n", xbar))
    cat(sprintf("sigma is %.2f\n", sigma))
    cat(sprintf("SEM is %.3f\n", SEM))
    cat(sprintf("The confidence interval is [%.3f,%.3f]", xbar-1.96*SEM, xbar+1.96*SEM))
}

In [15]:
confidence_interval(group_30$retention_1)

Number of samples is 44700
xbar is 0.45
sigma is 0.50
SEM is 0.002
The confidence interval is [0.444,0.453]

In [16]:
confidence_interval(group_40$retention_1)

Number of samples is 45489
xbar is 0.44
sigma is 0.50
SEM is 0.002
The confidence interval is [0.438,0.447]

In [17]:
confidence_interval(group_30$retention_7)

Number of samples is 44700
xbar is 0.19
sigma is 0.39
SEM is 0.002
The confidence interval is [0.187,0.194]

In [18]:
confidence_interval(group_40$retention_7)

Number of samples is 45489
xbar is 0.18
sigma is 0.39
SEM is 0.002
The confidence interval is [0.178,0.186]

In [19]:
confidence_interval(group_30$sum_gamerounds)

Number of samples is 44700
xbar is 52.46
sigma is 256.71
SEM is 1.214
The confidence interval is [50.076,54.836]

In [20]:
confidence_interval(group_40$sum_gamerounds)

Number of samples is 45489
xbar is 51.30
sigma is 103.29
SEM is 0.484
The confidence interval is [50.350,52.248]

Let's summarize our findings and make conclusions. Here is the table with all of the confidence intervals 

|             | `retention_1`   | `retention_7`       | `sum_gamerounds`  |
|-------------|---------------|-------------------|-----------------|
| gate_30     | [0.444,0.453] | [0.187,0.194]     | [50.076,54.836] |
| gate_40     | [0.438,0.447] | [0.178,0.186]     | [50.350,52.248] |
| Observation | Overlap       | No Overlap        | Overlap         |
| Decision    | No Difference | gate_30 is better | No Difference   |

&nbsp;
<center>
    <font size=6>The conclusion is</font>
</center>
&nbsp;
- <font color = red>`retention_1` `and sum_gamerounds` are not effected by change from `gate_30` to `gate_40`</font>
- <font color = green> but `retention_7` is worse for `gate_40` design</font>


# What have we learned?
- Basisc of R (arithmetic operations and data operations)
- Data manipulation techniques
- Random variables 
- Confidence intervals methodology
- We applied those skills to analyse data set from a mobile game to decide on optimal game design